In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

### Load model

In [ ]:
from in_silico.model.mlflow_loader import ModelPaths, DataPaths, load_free_viewing_model_from_mlflow

model_paths = ModelPaths(
    checkpoint_uri="mlflow-artifacts:/621818231566971674/2f85fd6f5dda46e280456d3186618e1c/artifacts/6806be20120f307fa684cd4c637ad949_final.pth.tar",
    config_uri="mlflow-artifacts:/621818231566971674/2f85fd6f5dda46e280456d3186618e1c/artifacts/config.yaml",
)

data_paths = DataPaths(
    base_dir="/gpfs01/euler/data/Data/",
)

out = load_free_viewing_model_from_mlflow(model_paths=model_paths, data_paths=data_paths)

In [ ]:
from in_silico.model.wrapper import ModelWrapper

model, skip_samples, cfg, extra = out

if skip_samples is None:
    skip_samples = cfg.trainer.skip_n_samples

wrapper = ModelWrapper(model=model, skip_samples=skip_samples)
print(f"skip_samples = {skip_samples}")
# With num_frames=12 and skip_samples=3, T_pred = 12 - 3 = 9 predicted frames.

### Load indices

In [ ]:
indices_v3a = np.load('/workdir/analysis_parametric/indices_v3a.npy')

### Create noise stimulus and preview

Each frame in the 12-frame batch is an independent random noise pattern. The noise is
block-based: every `block_size_px × block_size_px` region of the image shares one
random value. Three independent RGB channels are generated.

**Key parameters:**
- `block_size_px` – effective spatial resolution of the noise (larger → coarser)
- `noise_type` – `"white"` for i.i.d. binary (±1) blocks, `"pink"` for 1/f spatial correlations
- `num_frames` – must match the model's expected batch length (12)

In [ ]:
from in_silico.stimuli.white_noise import WhiteNoiseSpec, make_white_noise

spec = WhiteNoiseSpec(
    num_frames=12,
    fps=30.0,
    block_size_px=10,   # 10×10 px per noise unit
    noise_type="white", # "white" or "pink"
    contrast=1.0,
    mean_lum=0.5,
    seed=0,
    rgb=True,           # 3 independent colour channels
)

frames_example = make_white_noise(spec)  # (T=12, C=3, H=236, W=420)
print(f"frames shape: {frames_example.shape}  dtype: {frames_example.dtype}")
print(f"value range: [{frames_example.min():.3f}, {frames_example.max():.3f}]")

In [ ]:
# Preview: show 6 consecutive frames to confirm each frame is independent
fig, axes = plt.subplots(2, 6, figsize=(18, 6))
for t, ax in enumerate(axes.flat):
    ax.imshow(frames_example[t].transpose(1, 2, 0))  # (H, W, 3)
    ax.set_title(f"frame {t}")
    ax.axis("off")
plt.suptitle("White noise stimulus – one independent pattern per frame", y=1.01)
plt.tight_layout()
plt.show()

### Step-by-step: predict responses and compute stRF

The spatiotemporal RF (stRF) is estimated by **reverse correlation**:

$$
\text{stRF}[u, \tau, c, h, w] \;=\;
\frac{\sum_{n,t}\; r[n, u, t]\; s[n, t-\tau, c, h, w]}{\sum_{n,t}\; 1}
$$

where $r[n,u,t]$ is the predicted response of neuron $u$ at time $t$ for stimulus $n$,
and $s[n,t-\tau,c,h,w]$ is the stimulus frame $\tau$ steps before that response.

**Lag convention:** lag 0 = simultaneous stimulus–response; lag $k$ = stimulus
$k$ frames before the response. Larger lags reveal longer neural latencies.

**Frame budget with this model:**
- 12 input frames → 9 predicted frames (skip_samples = 3)
- With `n_lags=4` (lags 0–3) all 9 predicted frames contribute equally to every lag.

In [ ]:
from in_silico.analyses.strf_mapping import predict_responses_strf

N = 1000          # number of stimuli (increase for better SNR, e.g. 5000–10000)
batch_size = 10   # stimuli per forward pass
ds_factor = 4     # spatial downsampling of stored frames (4× → 59×105 px)

frames_all, pred_all, seeds = predict_responses_strf(
    wrapper,
    key="37_3843837605846_0_V3A_V4",
    spec=spec,
    N=N,
    batch_size=batch_size,
    ds_factor=ds_factor,
)

print(f"frames_all : {frames_all.shape}   (N, T, C, H_ds, W_ds)")
print(f"pred_all   : {pred_all.shape}     (N, U, T_pred)")

In [ ]:
from in_silico.analyses.strf_mapping import compute_spatiotemporal_sta

# skip_frames = num_frames - T_pred = 12 - 9 = 3
skip_frames = frames_all.shape[1] - pred_all.shape[2]
print(f"skip_frames = {skip_frames}")

strf = compute_spatiotemporal_sta(
    frames_all,
    pred_all,
    skip_frames=skip_frames,
    n_lags=4,          # lags 0, 1, 2, 3
    center_stim=True,
    center_resp=True,
    baseline=0.5,
)

print(f"strf shape : {strf.shape}   (U, n_lags, C, H_ds, W_ds)")

### Visualise stRF – one neuron, all lags

For each lag the spatial map shows which pixels drove the neuron:
- **Warm colours** (above grey) → ON response: the neuron preferred bright stimulation at that location.
- **Cool colours** (below grey) → OFF response.

The lag at which the map is sharpest indicates the neuron's preferred latency.

In [ ]:
from in_silico.analyses.strf_mapping import strf_to_rgb

# Select a neuron to inspect (change this index to browse different neurons)
neuron_idx = 0

strf_u = strf[neuron_idx]        # (n_lags, C, H_ds, W_ds)
lag_images = strf_to_rgb(
    strf_u,
    mode="robust",
    p=99.5,
    shared_scale=True,  # compare amplitudes across lags
)

fps = spec.fps
fig, axes = plt.subplots(1, len(lag_images), figsize=(4 * len(lag_images), 4))
if len(lag_images) == 1:
    axes = [axes]

for j, (img, ax) in enumerate(zip(lag_images, axes)):
    ax.imshow(img)
    ax.set_title(f"lag {j}  ({j / fps * 1000:.0f} ms)")
    ax.axis("off")

plt.suptitle(f"stRF – neuron {neuron_idx}", fontsize=14)
plt.tight_layout()
plt.show()

### Visualise stRF – multiple neurons

Each row is one neuron; each column is one lag.

In [ ]:
n_neurons_show = 6
n_lags_show = strf.shape[1]

# Pick neurons with the highest stRF power (summed across all lags and pixels)
strf_power = (strf ** 2).sum(axis=(1, 2, 3, 4))   # (U,)
top_neurons = np.argsort(strf_power)[::-1][:n_neurons_show]

fig, axes = plt.subplots(
    n_neurons_show, n_lags_show,
    figsize=(3.5 * n_lags_show, 3 * n_neurons_show),
)

for row, u in enumerate(top_neurons):
    lag_images = strf_to_rgb(strf[u], mode="robust", p=99.5, shared_scale=True)
    for col, img in enumerate(lag_images):
        ax = axes[row, col]
        ax.imshow(img)
        if row == 0:
            ax.set_title(f"lag {col}  ({col / spec.fps * 1000:.0f} ms)", fontsize=11)
        if col == 0:
            ax.set_ylabel(f"neuron {u}", fontsize=10)
        ax.axis("off")

plt.suptitle("stRF – top neurons by RF power", fontsize=14)
plt.tight_layout()
plt.show()

### Full pipeline (single call)

Once you are happy with the parameters, use `compute_strf` to run the
complete pipeline and optionally save the result.

In [ ]:
from in_silico.analyses.strf_mapping import compute_strf

strf_full = compute_strf(
    wrapper,
    key="37_3843837605846_0_V3A_V4",
    # --- stimulus ---
    num_frames=12,
    fps=30.0,
    block_size_px=10,
    noise_type="white",
    contrast=1.0,
    mean_lum=0.5,
    base_seed=0,
    rgb=True,
    N=1000,
    batch_size=10,
    ds_factor=4,
    # --- stRF ---
    skip_frames=3,     # = num_frames - T_pred
    n_lags=4,
    output_path="../results/strf_v3a.npy",  # set to None to skip saving
)

print(f"strf_full shape: {strf_full.shape}   (U, n_lags, C, H_ds, W_ds)")

### Compare white noise vs. pink noise

Pink noise has 1/f spatial correlations (more energy at low spatial frequencies),
which can improve SNR for neurons with large, smooth receptive fields.

In [ ]:
spec_pink = WhiteNoiseSpec(
    num_frames=12,
    fps=30.0,
    block_size_px=10,
    noise_type="pink",
    contrast=1.0,
    mean_lum=0.5,
    seed=0,
    rgb=True,
)

frames_pink = make_white_noise(spec_pink)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].imshow(frames_example[0].transpose(1, 2, 0))
axes[0].set_title("white noise (frame 0)")
axes[0].axis("off")

axes[1].imshow(frames_pink[0].transpose(1, 2, 0))
axes[1].set_title("pink noise (frame 0)")
axes[1].axis("off")

plt.tight_layout()
plt.show()